In [2]:
import numpy as np
from matplotlib import pyplot as plt
import torch
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
import torch.nn as nn
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

In [18]:
iris = load_iris()

In [19]:
X = iris['data']
y= iris['target']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 1./3, random_state =1)

In [20]:
def normalize(X):
    return (X - np.mean(X)) / np.std(X)

In [21]:
X_train_norm = torch.from_numpy(normalize(X_train)).float()


In [22]:
y_train = torch.from_numpy(y_train)

In [23]:
train_ds = TensorDataset(X_train_norm, y_train)

In [25]:
torch.manual_seed(1)
batch_size = 2
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

In [46]:
class Model(nn.Module):
    def __init__(self, input_size,hidden_size, output_size):
        super().__init__()
        self.layer1 = nn.Linear(input_size, hidden_size)
        self.layer2 = nn.Linear(hidden_size, output_size)

    def forward(self,x):
        x = self.layer1(x)
        x = nn.Sigmoid()(x)
        x = self.layer2(x)
        x = nn.Softmax(dim=1)(x)

        return x


In [47]:
input_size = X_train_norm.shape[1]
hidden_size = 16
output_size = 3
model = Model(input_size,hidden_size,output_size)

In [48]:
model

Model(
  (layer1): Linear(in_features=4, out_features=16, bias=True)
  (layer2): Linear(in_features=16, out_features=3, bias=True)
)

In [49]:
learning_rate = 0.001
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
num_epochs = 100
loss_hist = [0] * num_epochs
accuracy_hist = [0] * num_epochs
for epoch in range(num_epochs):
    for x_batch, y_batch in train_dl:
        pred = model(x_batch)
        loss = loss_fn(pred, y_batch)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        loss_hist[epoch] += loss.item()*y_batch.size(0)
        is_correct = (torch.argmax(pred, dim=1) == y_batch).float()
        accuracy_hist[epoch] += is_correct.mean()
    loss_hist[epoch] /= len(train_dl.dataset)
    accuracy_hist[epoch] /= len(train_dl.dataset)
    # if epoch % 10 == 0:
    #     print(f"Prediction is :: {pred}\n")
    #     print(f"y_batch is :: {y_batch}\n")
        


tensor(1.)